In [1]:
from cv_bridge import CvBridge
from rosbags.rosbag2 import Reader, Writer
from rosbags.typesys import Stores, get_typestore
import rosbags.image
from tqdm import tqdm

import numpy as np
import pandas as pd

import rospy.rostime
import cv2

import matplotlib.pyplot as plt

import os


import shutil
import math

from ultralytics import YOLO

import sqlite3

In [3]:
def fill_list(box_list: list, frame_rate:int=1, box_difference:int=5):
    """
    Modifies a list of lists (`box_list`) by filling in gaps and ensuring continuity between frames
    based on spatial proximity of bounding boxes.

    This function iterates through each list of boxes (representing frames) and checks for continuity 
    of each box between consecutive frames. If a box in a previous frame does not have a close match 
    in the current frame but has one in subsequent frames (within a specified `frame_rate`), a mean 
    box is created to bridge the gap.

    It will also add a box at before the first occurance of a box, for a more smooth blurring

    Parameters
    ----------
    box_list : list of list of list of float
        A list where each element is a list representing a frame of bounding boxes, and each bounding 
        box is a list of floats representing its coordinates.
    frame_rate : int, optional
        The number of frames to look ahead for matching a box from the previous frame, default is 1.
    box_difference : int, optional
        The allowed difference between the coordinates of boxes for them to be considered close, 
        default is 5.

    Returns
    -------
    list of list of list of float
        The modified `box_list` with added boxes to ensure continuity between frames.

    Examples
    --------
    >>> boxes = [
            [
                [10, 10, 20, 20],
                [20, 20, 40, 40]
            ],
            [
                [100, 100, 150, 150]
            ],
            [
                [20, 20, 40, 40]
            ],
            [
                [15, 15, 25, 25]
            ]
        ]
    >>> fill_list(boxes, frame_rate=3)
    [
        [
            [10, 10, 20, 20], 
            [20, 20, 40, 40], 
            [100, 100, 150, 150]        # was added 
        ],
        [
            [100, 100, 150, 150], 
            [12.5, 12.5, 22.5, 22.5],   # was added
            [20.0, 20.0, 40.0, 40.0]    # was added
        ],
        [
            [20, 20, 40, 40], 
            [13.75, 13.75, 23.75, 23.75]    # was added
        ],
        [
            [15, 15, 25, 25]
        ]
    ]
    """
    for i in range(1, len(box_list)-1):
        # on regarde les boxes de la frame précédente
        for last_boxes in box_list[i-1]:
            correspondance_now = False
            for present_boxe in box_list[i]:
                # on trouve une frame ressemblante dans la frame actuelle
                if np.isclose(last_boxes, present_boxe, atol=box_difference).all():
                    correspondance_now = True
            # si on trouve, alors on s'arrête là (pas besoin de créer de boxe)
            if correspondance_now:
                continue
            
            # on regarde si les frames d'après ressemble à une box de la frame précédente
            correspondance_after = False
            for j in range(frame_rate):
                if len(box_list) > i+j+1:
                    for next_boxe in box_list[i+j+1]:
                        # on trouve une frame ressemblante
                        if np.isclose(last_boxes, next_boxe, atol=box_difference).all():
                            correspondance_after = True
                            break
                    if correspondance_after:
                        break
            
            # si on trouve une frame ressemblante, alors on créer une approximation entre la
            # boxe de la frame précédente et suivante
            if correspondance_after:
                box_list[i].append(np.mean([last_boxes, next_boxe], axis=0).tolist())

        # on rajoute des boxes aux frames précédentes
        for present_boxe in box_list[i]:
            correspondance = False
            for last_boxe in box_list[i-1]:
                if np.isclose(last_boxe, present_boxe, atol=box_difference).all():
                    correspondance = True
            if not correspondance:
                for j in range(frame_rate):
                    if i-j-1 >= 0:
                        box_list[i-j-1].append(present_boxe)

    return box_list

In [4]:
def blur_box1(frame: np.ndarray, 
             box: list, 
             black_box: bool=False):
    """
    Apply a blur or black box to a specified region of an image if the confidence level is above a threshold.

    Parameters
    ----------
    frame : numpy.ndarray
        The image on which the blur or black box is to be applied. It should be a 3D array representing an RGB image.
    box : object
        An object containing the bounding box coordinates and confidence score. It should have attributes `conf` and `xyxy`:
        - `box.conf` : list or numpy.ndarray
            The confidence score(s) of the bounding box, with values between 0 and 1.
        - `box.xyxy` : list or numpy.ndarray
            The coordinates of the bounding box in the format [x1, y1, x2, y2].
    black_box : bool, optional
        If True, the specified region is filled with a black box instead of being blurred. Default is False.
    min_conf : float, optional
        The minimum confidence threshold to apply the blur or black box. Default is 0.3. Must in [0, 1].

    Returns
    -------
    numpy.ndarray
        The modified image with the blur or black box applied to the specified region.

    Notes
    -----
    - The input `frame` must be a 3D NumPy array representing an image with shape (height, width, channels).
    - The bounding box coordinates and confidence score must be provided in the `box` object (already implemented in the Yolov8 results).
    - If `black_box` is set to True, the region within the bounding box will be replaced with black pixels (faster than blurring).
    - The Gaussian blur applied uses a kernel size of (51, 51) with a standard deviation of 0.
    """
    x1, y1, x2, y2 = box
    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
    h, w = y2-y1, x2-x1

    if black_box:
        blur = blur = np.ones((h, w, 3)) * np.array([255, 0, 0])
    else:
        ROI = frame[y1:y1+h, x1:x1+w]
        blur = cv2.GaussianBlur(ROI, (51,51), 0) 
    frame[y1:y1+h, x1:x1+w] = blur
    return frame

In [5]:
bridge = CvBridge()
model = YOLO("models/yolov8n-face.pt")

In [6]:
INPUT_BAGFILE = '../../2022-11-17_pietons/Acquis2805_grise/'
OUTPUT_BAGFILE = 'output_ros2'

frame_rate = 5
min_conf = None

In [7]:
%%script false --no-raise-error
# cŕeation sortie
if not os.path.exists(OUTPUT_BAGFILE): os.mkdir(OUTPUT_BAGFILE)

db3_file_list = [file for file in os.listdir(OUTPUT_BAGFILE) if ".db" in file]
if len(db3_file_list) == 0:
    db3_file_name = [file for file in os.listdir(INPUT_BAGFILE) if len(file)> 1 and".db3" == file[-4:]][0]
    shutil.copyfile(os.path.join(INPUT_BAGFILE, db3_file_name), os.path.join(OUTPUT_BAGFILE, db3_file_name))
else:
    db3_file_name = db3_file_list[0]

yaml_file_list = [file for file in os.listdir(OUTPUT_BAGFILE) if ".yaml" in file]
if len(yaml_file_list) == 0:
    yaml_file_name = [file for file in os.listdir(INPUT_BAGFILE) if len(file)> 1 and".yaml" == file[-5:]][0]
    shutil.copyfile(os.path.join(INPUT_BAGFILE, yaml_file_name), os.path.join(OUTPUT_BAGFILE, yaml_file_name))

db3_path = os.path.join(OUTPUT_BAGFILE, db3_file_name)

db3_connection = sqlite3.connect(db3_path)
cursor = db3_connection.cursor()

In [8]:
%%script false --no-raise-error
topic_id = cursor.execute(r"SELECT id FROM topics WHERE name like '%image_raw%';").fetchone()[0]

In [9]:
%%script false --no-raise-error
# vérification chaque x frames
typestore = get_typestore(Stores.LATEST)

interesting_frames = []
# Create reader instance and open for reading.
with Reader('../../2022-11-17_pietons/Acquis2805_grise/') as reader:
    last_i = 0
    # Iterate over messages.
    for i, (connection, timestamp, rawdata) in enumerate(reader.messages()):
        if connection.topic == '/d435i/color/image_raw':
            last_rawdata = rawdata
            last_i = i
            if i%frame_rate == 0:
                
                msg = typestore.deserialize_cdr(rawdata, connection.msgtype)
                img = bridge.imgmsg_to_cv2(msg, desired_encoding="bgr8")
                
                boxes = next(model(img, stream=True, verbose=False)).boxes
                if len(boxes) > 0:
                    interesting_frames.append(True)
                else:
                    interesting_frames.append(False)
if last_i%frame_rate != 0:
    msg = typestore.deserialize_cdr(last_rawdata, connection.msgtype)
    img = bridge.imgmsg_to_cv2(msg, desired_encoding="bgr8")
    
    boxes = next(model(img, stream=True, verbose=False)).boxes
    if len(boxes) > 0:
        interesting_frames.append(True)
    else:
        interesting_frames.append(False)
# pd.DataFrame(interesting_frames).value_counts()

In [10]:
os.system("rm -Rf jsp")

0

In [11]:
with open("boxes_list") as file:
    boxes_list = eval(file.read())

In [ ]:
os.system("rm -Rf jsp")
# création boxes pour frames intérressantes et voisines
typestore = get_typestore(Stores.LATEST)
Image = typestore.types['sensor_msgs/msg/Image']


with Reader(INPUT_BAGFILE) as reader, Writer(OUTPUT_BAGFILE) as writer:
    last_imgs = []
    boxes_list = []
    begin = True
    save_imgs = False
    print(reader.message_count)
    for connection, timestamp, rawdata in tqdm(reader.messages()):
        if connection.topic == '/d435i/color/image_raw':
            topic_msg_type = connection.msgtype
            msg = typestore.deserialize_cdr(rawdata, connection.msgtype)
            topic_msg = msg
            img = bridge.imgmsg_to_cv2(msg, desired_encoding="bgr8").copy()
            last_imgs.append(img)

            if begin and len(last_imgs) == math.ceil(frame_rate/2):
                begin = False
                save_imgs = True
                idx_img = 0
            elif len(last_imgs) == frame_rate:
                save_imgs = True
                idx_img = math.ceil(frame_rate/2)-1

            if save_imgs:
                save_imgs = False
                boxes = next(model(last_imgs[idx_img], stream=True, verbose=False)).boxes
                to_blur = len(boxes) > 0
                for img in last_imgs:
                    boxes_list.append([])
                    if to_blur:
                        boxes = next(model(img, stream=True, verbose=False)).boxes
                        if len(boxes.conf) > 0:
                            if isinstance(min_conf, type(None)):
                                min_conf = boxes.conf.mean() * 0.6
                            else:
                                min_conf = (min_conf + boxes.conf.mean())/2 *0.6
                        for box in boxes:
                            if box.conf[0] > min_conf:
                                boxes_list[-1].append(box.xyxy[0].tolist())
                last_imgs = []
    # dernières frames
    boxes = next(model(last_imgs[-1], stream=True, verbose=False)).boxes
    to_blur = len(boxes) > 0
    for img in last_imgs:
        boxes_list.append([])
        if to_blur:
            boxes = next(model(img, stream=True, verbose=False)).boxes
            
            for box in boxes:
                boxes_list[-1].append(box.xyxy[0].tolist())

    boxes_list = fill_list(boxes_list, frame_rate*2, math.ceil(math.sqrt(max(msg.width, msg.height)))*2.5)

    # fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    # video = cv2.VideoWriter(filename="supp.mp4", 
    #                     fourcc=fourcc, 
    #                     fps=15, 
    #                     frameSize=(topic_msg.width, topic_msg.height))

    idx_boxes = 0
    writer_connection = writer.add_connection("/d435i/color/image_raw", 'sensor_msgs/msg/Image', typestore=typestore)
    for connection, timestamp, rawdata in tqdm(reader.messages()):
        if connection.topic == "/d435i/color/image_raw":
            # msg = typestore.deserialize_cdr(rawdata, connection.msgtype)
            # img = bridge.imgmsg_to_cv2(msg, desired_encoding="bgr8").copy()
            # for box in boxes_list[idx_boxes]:
            #     img = blur_box1(img, box)
            # video.write(img)
            if len(boxes_list[idx_boxes]) > 0:
                msg = typestore.deserialize_cdr(rawdata, connection.msgtype)
                img = bridge.imgmsg_to_cv2(msg, desired_encoding="rgb8").copy()
                for box in boxes_list[idx_boxes]:
                    img = blur_box1(img, box)

                image_message = bridge.cv2_to_imgmsg(img, encoding="rgb8")
                image_message = Image(
                    image_message.header,
                    image_message.height,
                    image_message.width,
                    image_message.encoding,
                    image_message.is_bigendian,
                    image_message.step,
                    image_message.data
                )
                new_msg = typestore.serialize_cdr(image_message, 'sensor_msgs/msg/Image')
                writer.write(writer_connection, timestamp, new_msg)
                # sql = f"UPDATE messages SET data=? WHERE timestamp={timestamp} AND topic_id={topic_id}"
                # cursor.execute(sql, (image_message, ))
                # db3_connection.commit()

            idx_boxes += 1

# video.release()
os.system("rm -Rf jsp")

92566


92566it [02:41, 571.69it/s] 
218it [00:00, 8386.07it/s]


AttributeError: 'Time' object has no attribute 'sec'

In [ ]:
msg.width, msg.height

(1280, 720)

In [ ]:
with open("boxes_list", "w") as file:
    file.write(str(boxes_list))

In [ ]:
db3_connection.close()

NameError: name 'db3_connection' is not defined

In [ ]:
len(boxes_list)

6971

In [ ]:
type(image_message)

sensor_msgs.msg._Image.Image

In [ ]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(filename="supp.mp4", 
                    fourcc=fourcc, 
                    fps=15, 
                    frameSize=(msg.width, msg.height))

with Reader(OUTPUT_BAGFILE) as reader:
    print(reader.message_count)
    # Iterate over messages.
    for connection, timestamp, rawdata in tqdm(reader.messages()):
        if connection.topic == '/d435i/color/image_raw':
            msg = typestore.deserialize_cdr(rawdata, connection.msgtype)
            video.write(bridge.imgmsg_to_cv2(msg, desired_encoding="bgr8"))
video.release()

92566


384it [00:00, 1530.65it/s]


TypeError: a bytes-like object is required, not 'str'

In [ ]:
db3_connection = sqlite3.connect(db3_path)
cursor = db3_connection.cursor()

In [ ]:
image_message._get_types()

['std_msgs/Header', 'uint32', 'uint32', 'string', 'uint8', 'uint32', 'uint8[]']

In [ ]:
typestore = get_typestore(Stores.LATEST)
with Writer("output_ros2_2") as writer, Reader(INPUT_BAGFILE) as reader:
    topic = '/chatter'
    msgtype = String.__msgtype__
    connection = writer.add_connection(topic, msgtype, typestore=typestore)

    # Serialize and write message.
    timestamp = 42
    message = String('hello world')
    writer.write(connection, timestamp, typestore.serialize_cdr(message, msgtype))

In [ ]:
from rosbags.typesys import Stores, get_typestore

typestore = get_typestore(Stores.ROS2_FOXY)


In [ ]:
typestore.types

{'action_msgs/msg/GoalInfo': rosbags.typesys.stores.ros2_dashing.action_msgs__msg__GoalInfo,
 'action_msgs/msg/GoalStatus': rosbags.typesys.stores.ros2_dashing.action_msgs__msg__GoalStatus,
 'action_msgs/msg/GoalStatusArray': rosbags.typesys.stores.ros2_dashing.action_msgs__msg__GoalStatusArray,
 'actionlib_msgs/msg/GoalID': rosbags.typesys.stores.ros2_dashing.actionlib_msgs__msg__GoalID,
 'actionlib_msgs/msg/GoalStatus': rosbags.typesys.stores.ros2_dashing.actionlib_msgs__msg__GoalStatus,
 'actionlib_msgs/msg/GoalStatusArray': rosbags.typesys.stores.ros2_dashing.actionlib_msgs__msg__GoalStatusArray,
 'builtin_interfaces/msg/Duration': rosbags.typesys.stores.empty.builtin_interfaces__msg__Duration,
 'builtin_interfaces/msg/Time': rosbags.typesys.stores.empty.builtin_interfaces__msg__Time,
 'diagnostic_msgs/msg/DiagnosticArray': rosbags.typesys.stores.ros2_dashing.diagnostic_msgs__msg__DiagnosticArray,
 'diagnostic_msgs/msg/DiagnosticStatus': rosbags.typesys.stores.ros2_dashing.diagnost